<a href="https://colab.research.google.com/github/gcosma/IEEE_Access_mHealth/blob/master/IEEEAccessmMHealth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

              **Click on the OPEN IN COLAB icon above**

**Feature Extraction and Classification using Leading Eigenvectors: Applications to Biomedical and Multi-Modal mHealth Data** 

by Dr Georgina Cosma

**********************************************************************************************************************************************
DESCRIPTION: This is supplementary code for the paper "Feature Extraction and Classification using Leading Eigenvectors: Applications to Biomedical and Multi-Modal mHealth Data" submitted to IEEE Access. 

Programming Language: This code was written in the Python programming language. CoLab is used for wider accessibility of the code.

File name: IEEEAccessmMHealth.ipynb

CONTACT INFORMATION: 
Dr Georgina Cosma, Department of Computer Science, Loughborough University.

E-mail: gcosmaresearch@outlook.com

Website: https://gcosma.home.blog/

PAPER CITATION: Please cite:
Cosma and McGinnity, 2019. Feature Extraction and Classification using Leading Eigenvectors: Applications to Biomedical and Multi-Modal mHealth Data. IEEE Access. 
**********************************************************************************************************************************************

Learning outcomes:
* Load Data.
* Apply z-score data normalisation
* Get the frequency of each class 
* Visualise the class labels
* Apply Singular Value Decomposition and extract the leading Eigenvectors
* Define a Sequential model for detecting human activity in multi-sensor data
* Compile the Sequential Model
* Train and Test a mode using k-fold cross validation.  


**Not using Colab?** If you are not using Colab you will need to Setup a Python Environment for 
Machine Learning and Deep Learning with Anaconda. You must have Python 2 or 3 
installed and configured. You must install SciPy (including NumPy) and the relevant
libraries including Keras. 

**Using Colab:** Some difficulties may be
experienced with mounting, but the code and explanation here will help you overcome these. 

I have included the datasets  in the GitHub repository and you can also download the data from: https://archive.ics.uci.edu/ml/datasets/MHEALTH+Dataset

**About the mHealth multi-sensor dataset:** The mHealth (Mobile Health) dataset is a benchmark dataset for human behaviour analysis based on multi-modal body sensing. The mHealth dataset comprises body motion and vital signs recordings for ten volunteers of diverse profile while performing 12 physical activities: Standing still (1 min), Sitting and relaxing (1 min), Lying down (1 min), Walking (1 min), Climbing stairs (1 min), Waist bends forward (20x), Frontal elevation of arms (20x), Knees bending (crouching) (20x), Cycling (1 min), Jogging (1 min), Running (1 min), and Jump front & back (20x). Sensors on each subject's chest, right wrist and left ankle were used to measure the motion experienced by diverse body parts, namely, acceleration, rate of turn and magnetic field orientation. All sensing modalities are recorded at a sampling rate of 50 Hz, which is considered sufficient for capturing human activity. This dataset has been found to generalize to common activities of the daily living, due to the diversity of body parts involved in each activity (e.g., frontal elevation of arms vs. knees bending), the intensity of the actions (e.g., cycling vs. sitting and relaxing) and their execution speed or dynamicity (e.g., running vs. standing still). For more information see: https://archive.ics.uci.edu/ml/datasets/MHEALTH+Dataset




**Step 1: Import libraries** 

In [0]:
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Activation
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from scipy import stats
#Below several optimisers are imported. This tutorial is using the Adam optimiser
#but you can easily replace Adam with another optimiser from the list. 
from keras.optimizers import SGD, Adam, Adadelta, RMSprop, Adagrad, Adamax, Nadam

**Step 2: Mount to Google Drive in order to access your data file**

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/Colab Notebooks"

#if you need to remount
#drive.mount("/content/drive", force_remount=True)

#If you want to unmount and reset then: 
#Step 1: From the menu select Runtime--->Reset all Runtimes... 
#Step 2: Runtime--->Run all or you can run each Cell at a time. There will be a message 
# "Go to a URL in a browser" and you must click on that and copy and paste the authorisation code 
# from the page into the authorisation code text box.

#! git clone https://github.com/gcosma/DeepLearningTutorials/DataTest
#! git clode https://github.com/gcosma/Data
#! ls

**Step 3: Choose your dataset. Each mHealth dataset belings to one person. There are 10 mHealth datasets.**

In [0]:
#All datasets are available in Github. I added the datasets into a data folder where all data are kept separate from the code. 
# To load the dataset you must uncomment the dataset you want to load. One dataset contains the data of a single person. 
dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject1txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject2txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject3txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject4txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject5txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject6txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject7txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject8txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealthdata/MHEALTHDATASET/mHealth_subject9txt.csv", delimiter=",")
#dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/MHEALTHDATASET/mHealth_subject10txt.csv", delimiter=",")

dataset.shape

**Step 4:**
Dataset comprises of 23 inputs, and a vector of categorical target values (0-12). Each target value corresponds to one of 12 physical activities as explained in the first part of this tutorial. Class 0 is not an activity. Hence, there are 13 classes but 12 activities. 

In [0]:
#Input data and labels need to be separated before training the model.
X = dataset[:,0:23]        
labels= dataset[:,23]

#Here zscore is applied to normalise the values of each column. 
# Z-scores are linearly transformed data values having a mean of zero and a standard deviation of 1. 
X=stats.zscore(X)

#Convert the single vector of 13 classes to a matrix of 1s and 0s. 
#This is called one-hot-encoding. With one-hot encoding the integer encoded variable is removed and a new binary 
# variable is added for each unique integer value. For a quick explanation see: https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/
Yenc = to_categorical(labels) 
print(Yenc)

#Check the size of matrix Yenc. The number of rows should be equal to the number of classes classes.
#In this example classes start from 0 to 12. Therefore we have 13 classes. 
Yenc.shape
#X.shape
#print(X)

**Count the frequency of each class. The datasets are considered as Limited Training Datasets making this classification task a challenging one. The reason the datasets are considered to be limited is because the cases in the null class range from 65.73\%-78.19\%, whereas the cases in the remaining 12 classes range from 0.67\%-3.13\%. Hence, the number of cases in all classes 0-12 are comparatively lower than the 0 class, and the number of training samples are limited.**

In [0]:
#Count the number of cases in each class. 
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))


**Plot a histogram of the labels**

In [0]:
plt.hist(labels, bins=np.arange(labels.min(), labels.max()+1))
plt.title("Histogram with 'auto' bins")

**Below is the code for saving the normalised data into a csv file.**





In [0]:
from pandas import DataFrame. For more information on DataFrames see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html
#create a Pandas DataFrame for matrix X
df = DataFrame(X)
#Export the DataFrame to a csv file and store it in a directory. 
export_csv = df.to_csv(r'/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/ProcessedMHEALTHDATASET/D1norm.csv', header=False, index = None)
print (export_csv)

#load the normalised csv file into a matrix
datasetnorm = np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/ProcessedMHEALTHDATASET/D1norm.csv", delimiter=",")
print(datasetnorm)

**Step 5: Apply Dingular Value Decomposition and Dimensionality reduction.**
A Sparse SVD method was applied. the k number of dimensions selected were 10, hence k=10

In [0]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, eigs
u, s, vt = svds(datasetnorm, k=10) # apply svds with k=10 dimensions
#print(u) # this is the patient by dimension matrix, u
u.shape

In [0]:
#Create a panda DataFrame for storing the u patient-by-dimension matrix
dfu = DataFrame(u)
#Export to a csv file
export_u = dfu.to_csv(r'/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/ProcessedMHEALTHDATASET/D1normU.csv', header=False, index = None)

#Load the normalised csv file into a matrix
U=np.loadtxt("/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/data/ProcessedMHEALTHDATASET/D1normU.csv", delimiter=",")
#print(U) 

**Step 6: Create the Sequential model**
A Sequential model is a linear stack of layers.
Code for the k-fold cross validation was adapted from https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

fit(X,Y) is for training the model with the given inputs X (and corresponding training labels Y). Adjust the number of epochs to avoid overtraining your network.

evaluate(X,Y) is for evaluating the already trained model using the test data. Returns the accuracy values for each k-fold using the test data. **



In [0]:
# Define k-fold cross validation test. In this application the number of folds is 10, hence n_splits=10.
# 10-fold allows us to partition the data into k different training and testing datasets. 
# k-fold, when applied to large datasets, can result in slow training, but it is a suitable 
# validation approach when the datasets are limited (see step 4 for an explanation of what is meant by 'limited'). 
# k-fold results to thorough evaluations of the model.  
# For a nice introduction to k-fold cross validation see: https://machinelearningmastery.com/k-fold-cross-validation/
kfold = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
cvscores = []
#get number of columns in training data
n_cols = U.shape[1]
#Split the dataset into folds of train and test data.
for train, test in kfold.split(U, labels):
  model = Sequential()
  # Dense(256) is a fully-connected layer with 256 hidden units.
  # in the first layer, you must specify the expected input data shape:
  # here, n_cols=23-dimensional vectors. n_cols = U.shape[1] gave 
  # the number of columns in the data. 
  # For more information on the options for constructing a Sequential model see
  # https://keras.io/getting-started/sequential-model-guide/ 
  model.add(Dense(256, activation='relu', input_shape=(n_cols,), name='input_layer'))
  model.add(Activation('relu', name='Activation_1'))
  #model.add(Dropout(0.5))
  model.add(Dense(256, activation='relu', name='Dense_1'))
  model.add(Activation('relu', name='Activation_2'))
  model.add(Dropout(0.5, name='Dropout_1'))
  model.add(Dense(256,activation='relu', name='Dense_2'))
  #13 refers to the number of classes. SOFTMAX is used for multi-classification.
  model.add(Dense(13, activation='softmax', name='Dense_3'))

  #####################################################################################
  #Setup the optimisers - you can try out several one at a time 
  # simply relace optimizer='adam' when you compile the model. 
  sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
  adam=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  adadelt=Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
  rmsprop=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
  adagrad=Adagrad(lr=0.01, epsilon=None, decay=0.0)
  adamax=Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
  nadam=Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
  #####################################################################################

  ############### Now you can change the name of the optimiser if you wish. 
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  ############# Fit the model (training)
  model.fit(U[train], Yenc[train], epochs=20, batch_size=50, verbose=0)

  ###################### Test the model using the test data
  score = model.evaluate(U[test], Yenc[test], batch_size=50, verbose=0)
  print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
  #Displays the accuracy of each fold when applied to the test data.
  cvscores.append(score[1] * 100)
  #Indentation is important here. Show the mean accuracy and standard deviation over all k-folds
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [0]:
# Print the structure of the Sequential model
print(model.summary())

**Extra code. How to install the GraphViz Library  http://www.graphviz.org/
The image is stored in your drive.**



In [0]:
!pip install -q pydot
from keras.utils.vis_utils import plot_model
#Prints the model to a file in the directory
plot_model(model, to_file='/content/drive/My Drive/Colab Notebooks/CosmaIEEEAccessPapermHealth/model_plot3.png', show_shapes=True, show_layer_names=True)

In [0]:
#install the dataframe panda
!pip install dataframe

**Extra code. How to decode a one-hot-encoded matrix**

In [0]:
#Decode from one-hot-encoded matrix
res= [np.where(r==1)[0][0] for r in Yenc]
print(res)
max(res)